# Clustering algorithms

Unsupervised learning I

Author: Emily Hunt

## Imports & dependencies

In [ ]:
# Uncomment the below to install packages you will need:
# !pip install scikit-learn numpy<2.0 pandas astropy astroquery

In [ ]:
import numpy as np
import pandas as pd
import sklearn

x = 4